#### Internet speed

In [ ]:

#@markdown

!sudo apt install speedtest-cli
!speedtest-cli --secure

#### Setup

In [ ]:

#@markdown
#!pip3 install requests &> /dev/null

import os  
from google.colab import drive
import ipywidgets as widgets
from IPython.display import display

Drive = True #@param {type:"boolean"}
Ftp = True #@param {type:"boolean"}

if Drive and not os.path.exists("/content/drive"):
  drive.mount('/content/drive')

if Ftp and not 'ftputil' in globals():
  !pip install ftputil &> /dev/null
  import ftputil

Host = "46.101.177.171" #@param {type: "string"}
Username = "dhamantr" #@param {type: "string"}
Password = "" #@param {type: "string"}

def data_str(byte_val):
  size = ByteToGB(byte_val)
  if (size >= 1) :
    return "{:.1f} GB".format(size)
  size = ByteToMB(byte_val)
  if (size >= 1) :
    return "{:.1f} MB".format(size)
  size = ByteToKB(byte_val)
  if (size >= 1) :
    return "{:.1f} KB".format(size)
  return "{:.1f} B".format(size)

def speed_str(byte_val):
  size_mb = ByteToMB(byte_val)
  if (size_mb >= 1) :
    return "{:.1f} MB".format(size_mb)
  else:
    return "{:.1f} KB".format(ByteToKB(byte_val))
    
def RoundTo1(n) :
  if n < 1 and (n - 0.9765625) >= 0 :
    return 1
  else: 
    return n

ByteToGB = lambda n: RoundTo1(n / 1073741824)
ByteToMB = lambda n: RoundTo1(n/ 1048576)
ByteToKB = lambda n: RoundTo1(n/ 1024)
contains = lambda string, search: search.lower() in string.lower()
time_str = lambda sec: time.strftime('%H:%M:%S', time.gmtime(sec))

size_limit = 40000

def has_size(path, only_check = True):
  size = 0
  if not os.path.exists(path):
    return 0
  if os.path.isfile(path):
    size = os.path.getsize(path)
  else:  
    for path, dirs, files in os.walk(path):
        for f in files:
            fp = os.path.join(path, f)
            size += os.path.getsize(fp)
            if only_check and size >= size_limit:
              return size
  if size >= size_limit:
    return size
  else:
    return 0

def is_hidden(path: str):
  return True if path[0] == '.' else False

#### Scrape

In [ ]:

#@markdown

from bs4 import BeautifulSoup
import requests
import json
import time

Base = "" #@param {type:"string"}
Target = "" #@param {type:"string"}
Limit = 0 #@param {type:"number"}

def scrape_posts(link_base, limit = 0, target_class = '', id_var = ''):
  limit = 100 if limit == 0 else limit
  var = f"?{id_var}=" if id_var else ''
  scrape = {}
  posts = []
  for i in range(limit + 1):
    time.sleep(0.1)
    url = f"{link_base}/?{var}{i}"
    response = requests.get(url)
    if response.status_code == 200:
      soup = BeautifulSoup(response.text, 'html.parser')
      page = soup.find('div', {"class": target_class})
      if page:
        print(f'Post found: {url}')
        posts.append(str(page))
        continue
    print(f'No Post:   {url}')

  if posts:
    scrape['posts'] = posts
    return json.dumps(scrape)
  else:
    print('No posts found.')
    return ''

def write_json(text, filename = 'scrape_output.json'):
  with open(filename, 'w') as file:
    file.write(text)

def read_json(filename):
  with open(filename, 'r') as file:
    json_dict = json.load(file)    
    return json_dict

scrape = scrape_posts(Base, target_class = Target, limit = Limit)
write_json(scrape)

#### FTP

In [ ]:


#@markdown

import re
Dir = "" #@param {type:"string"}
Size = False #@param {type:"boolean"}
file_re = r"^.*\.[A-Z,a-z,0-9]+$"

def ls(dir = "", size = False):
  total_size = 0
  with ftputil.FTPHost(Host, Username, Password) as ftp:
    if not dir:
      dir = ftp.curdir
    names = ftp.listdir(dir)
    names.sort(key = lambda obj: bool(re.search(file_re, obj)))
    for name in names:
      if size:
        p0 = dir + '/' + name
        s0 = 0
        if ftp.path.isfile(p0):
          s0 = ftp.path.getsize(p0)
        else:  
          for path, dirs, files in ftp.walk(p0):
              for f in files:
                  fp = path + '/' + f
                  s0 += ftp.path.getsize(fp)
        item = "{:>6}  {}".format(data_str(s0), name)
        total_size += s0
      else:
        item = name
      print(item)
    if total_size:
      print(f"\nTotal size: {data_str(total_size)}")
ls(Dir, Size)


#### Transfer

In [ ]:

#@markdown

Upload = 0
Download = 1

Path = "" #@param {type: "string"}
SavePath = '/content/drive/Shareddrives/SM50/Websites/Dhaman' #@param {type: "string"}
Action = Download #@param ["Upload", "Download"] {type: "raw"}
done_path = '/content/done'
os.makedirs(done_path, exist_ok = True)
os.makedirs(SavePath, exist_ok = True)
total_size = 0
total_str = ""
done = 0
patch_size = 0

layout_15 ={'width':'auto', 'flex':'15 1 0%', 'align_items':"center"}
layout_10 ={'width':'auto', 'flex':'10 1 0%', 'align_items':"center"}
layout_8 ={'width':'auto', 'flex':'8 1 0%', 'align_items':"center"}
layout_7 ={'width':'auto', 'flex':'7 1 0%', 'align_items':"center"}
layout_5 ={'width':'auto', 'flex':'5 1 0%', 'align_items':"center"}
layout_4 ={'width':'auto', 'flex':'4 1 0%', 'align_items':"center"}
layout_3 ={'width':'auto', 'flex':'3 1 0%', 'align_items':"center"}
layout_2 ={'width':'auto', 'flex':'2 1 0%', 'align_items':"center"}
layout_1 ={'width':'auto', 'flex':'1 1 0%', 'align_items':"center"}
layout_r = {'padding-left':'100px','flex_flow':'row','align_items':'flex-start','width':'95%','justify_content':'space-around'}
style0= {"padding-left":"200px","padding-right":"200px"}

empty_label_1 = widgets.Label(value = "", layout = layout_1)
empty_label_2 = widgets.Label(value = "", layout = layout_2)
empty_label_3 = widgets.Label(value = "", layout = layout_3)
empty_label_4 = widgets.Label(value = "", layout = layout_4)
empty_label_5 = widgets.Label(value = "", layout = layout_5)

progress_bar = widgets.FloatProgress(
      value=0,
      min=0,
      max=100,
      step=0.1,
      bar_style='info',
      orientation='horizontal',
      layout=layout_15)

done_label = widgets.Label(
    value = f"",
    layout = layout_2
)

progress_label = widgets.Label(
    value = f"0%",
    layout = layout_1
)

download_label = widgets.Label(
    value = f"Downloading",
    layout = layout_r
)

r1 = widgets.Box([ empty_label_2, done_label, empty_label_1, progress_label, progress_bar, empty_label_2], layout = layout_r, style = style0)

def update_upload(chunk):
  global done, total_size, progress_bar, progress_label, done_label
  done += len(chunk)
  progress = round(done*100/total_size, 1)
  progress_bar.value = progress
  if progress == 100:
    progress_bar.bar_style = 'success'
  done_label.value = f"{data_str(done)} {total_str}"
  progress_label.value = f"{progress}%"

def update_download(chunk):
  global done, download_label, patch_size 
  patch = len(chunk)
  done += patch
  result = ' -  Transfered Successfully' if patch < patch_size else ''
  download_label.value = f"Transfered {data_str(done)} {result}"
  if not patch_size:
    patch_size = patch

def upload(path, savepath, callback = None):
  global total_size, total_str
  total_size = os.path.getsize(Path)
  total_str = data_str(total_size)
  with ftputil.FTPHost(Host, Username, Password) as ftphost:
    ftphost.upload_if_newer(path, savepath, callback = callback)

def download(path, savepath, callback = None):
  global total_size, total_str
  total_size = 0
  total_str = ''
  with ftputil.FTPHost(Host, Username, Password) as ftp:
    target = f"{done_path}/{os.path.basename(path)}"
    ftp.download(path, target, callback = callback)
    if has_size(target):
      !mv "$target" "$savepath"

if Path:
  if Action == Upload:
    display(r1)
    upload(Path, SavePath, callback = update_upload)
  else:
    display(download_label)
    download(Path, SavePath, callback = update_download)



---


In [ ]:


#@markdown
# codex

import ftputil

# Download some files from the login directory.
with ftputil.FTPHost("ftp.domain.com", "user", "password") as ftphost:
  names = ftphost.listdir(ftphost.curdir)
  for name in names:
      if ftphost.path.isfile(name):
          ftphost.download(name, name)  # remote, local

  # Make a new directory and copy a remote file into it.
  ftphost.mkdir("newdir")
  with ftphost.open("index.html", "rb") as source:
      with ftphost.open("newdir/index.html", "wb") as target:
          ftphost.copyfileobj(source, target)  # similar to shutil.copyfileobj

  ftphost.upload(source, target, callback=None)
  ftphost.upload_if_newer(source, target, callback=None)